In [9]:
# Bitcoin Analysis and Visualization Script - Version 003
# This version includes bar charts without outlines for metrics visualization
# and a heatmap for overall sentiment analysis

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2023, 1, 1).timestamp())  # Jan 1, 2023
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/spot_cvd_sum',
    'https://api.glassnode.com/v1/metrics/market/spot_volume_daily_sum'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

def calculate_momentum_rsi(df, column='price_usd_close', rsi_window=14, window_norm=90, normalize=True):
    price_change = df[column].diff()
    gains = price_change.where(price_change > 0, 0)
    losses = -price_change.where(price_change < 0, 0)
    avg_gains = gains.rolling(window=rsi_window, min_periods=1).mean()
    avg_losses = losses.rolling(window=rsi_window, min_periods=1).mean()
    relative_strength = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + relative_strength))

    if normalize:
        rsi_min = rsi.rolling(window=window_norm, min_periods=1).min()
        rsi_max = rsi.rolling(window=window_norm, min_periods=1).max()
        normalized_momentum = 2 * (rsi - rsi_min) / (rsi_max - rsi_min) - 1
        return normalized_momentum
    else:
        return rsi

def calculate_spot_cvd_bias(df, column='spot_cvd_sum', window_sum=7, window_norm=90, normalize=True):
    rolling_sum = df[column].rolling(window=window_sum).sum()
    
    if normalize:
        rolling_min = rolling_sum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = rolling_sum.rolling(window=window_norm, min_periods=1).max()
        normalized_bias = 2 * (rolling_sum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_bias
    else:
        return rolling_sum

def calculate_spot_volume_momentum(df, column='spot_volume_daily_sum', fast_window=7, slow_window=90, window_norm=90, normalize=True):
    fast_ma = df[column].rolling(window=fast_window).mean()
    slow_ma = df[column].rolling(window=slow_window).mean()
    volume_momentum = fast_ma / slow_ma

    if normalize:
        rolling_min = volume_momentum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = volume_momentum.rolling(window=window_norm, min_periods=1).max()
        normalized_momentum = 2 * (volume_momentum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_momentum
    else:
        return volume_momentum

# Apply the functions to our merged_df
merged_df['Price Momentum'] = calculate_momentum_rsi(merged_df, column='price_usd_close', rsi_window=14, window_norm=90, normalize=True)
merged_df['Spot CVD Bias'] = calculate_spot_cvd_bias(merged_df, column='spot_cvd_sum', window_sum=7, window_norm=90, normalize=True)
merged_df['Spot Volume Momentum'] = calculate_spot_volume_momentum(merged_df, column='spot_volume_daily_sum', fast_window=7, slow_window=90, window_norm=90, normalize=True)

def create_chart(df, metric_name, chart_title):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add bar chart for the metric
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=df[metric_name],
            name=metric_name,
            marker_color=df[metric_name].apply(lambda x: 'rgba(0,255,0,0.6)' if x >= 0 else 'rgba(255,0,0,0.6)'),
            marker_line_width=0  # This removes the outline of the bars
        ),
        secondary_y=False,
    )

    # Add trace for BTC price
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['price_usd_close'],
            mode='lines',
            line=dict(color='gray', width=1),
            name='BTC price in $'
        ),
        secondary_y=True,
    )

    # Configure layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'grey'}
        },
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            title='Date',
            titlefont={'color': 'grey'},
            showgrid=False,
            showline=True,
            linewidth=1,
            linecolor='grey',
            ticks='outside',
            ticklen=5,
            tickwidth=1,
            tickcolor='grey',
            tickfont={'color': 'grey'},
            tickformat='%b %y',
            tickmode='auto',
            nticks=10,
            mirror=True
        ),
        yaxis=dict(
            title=metric_name,
            titlefont={'color': 'grey'},
            range=[-1, 1],
            showgrid=False,
            zeroline=True,
            zerolinecolor='rgba(0,0,0,0.2)',
            tickmode='array',
            tickvals=[-1, -0.5, 0, 0.5, 1],
            ticktext=['-1', '-0.5', '0', '0.5', '1'],
            ticks='outside',
            ticklen=5,
            tickwidth=1,
            tickcolor='grey',
            tickfont={'color': 'grey'},
            showline=True,
            linewidth=1,
            linecolor='grey',
            mirror=True
        ),
        yaxis2=dict(
            title='BTC price in $',
            titlefont={'color': 'grey'},
            showgrid=False,
            showline=True,
            linewidth=1,
            linecolor='grey',
            ticks='outside',
            ticklen=5,
            tickwidth=1,
            tickcolor='grey',
            tickfont={'color': 'grey'},
            mirror=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1,
            font={'color': 'grey'}
        ),
        hovermode='x unified',
        barmode='relative'  # This ensures the bars are centered on zero
    )

    return fig

def create_heatmap(df):
    indicators = ['Price Momentum', 'Spot CVD Bias', 'Spot Volume Momentum']
    
    fig = go.Figure(data=go.Heatmap(
        z=[df[indicator] for indicator in indicators],
        x=df.index,
        y=indicators,
        colorscale=[
            [0, 'red'],      # Low risk (-1)
            [0.5, 'yellow'], # Moderate risk (0)
            [1, 'green']     # High risk (+1)
        ],
        zmin=-1,
        zmax=1,
        colorbar=dict(
            title='Risk Level',
            tickvals=[-1, 0, 1],
            ticktext=['Low Risk', 'Moderate Risk', 'High Risk']
        )
    ))

    fig.update_layout(
        title='Spot Market Sentiment Heatmap',
        xaxis_title='Date',
        yaxis_title='Indicators',
        height=600,
        yaxis=dict(
            tickmode='array',
            tickvals=[0, 1, 2],
            ticktext=indicators
        )
    )

    return fig

# Create and display the three charts
charts = [
    ("Price Momentum", "Bitcoin Price and Price Momentum"),
    ("Spot CVD Bias", "Bitcoin Price and Spot CVD Bias"),
    ("Spot Volume Momentum", "Bitcoin Price and Spot Volume Momentum")
]

for metric, title in charts:
    fig = create_chart(merged_df, metric, title)
    fig.show()

# Create and display the heatmap
heatmap = create_heatmap(merged_df)
heatmap.show()

In [10]:
merged_df.tail()

,price_usd_close,spot_cvd_sum,spot_volume_daily_sum,Price Momentum,Spot CVD Bias,Spot Volume Momentum
t,,,,,,
2024-08-31,58971.654336,-2.149215e+07,2.178212e+09,-0.066982,-0.389954,-0.396320
2024-09-01,57319.948544,-2.361100e+08,5.535664e+09,-0.122606,-0.615546,-0.386101
2024-09-02,59099.379977,9.461362e+06,6.223415e+09,-0.063232,-0.431569,-0.433198
2024-09-03,57469.422699,-1.274605e+08,6.259370e+09,-0.144234,-0.290310,-0.552043
2024-09-04,57982.587216,1.339837e+07,9.142887e+09,-0.287697,-0.069403,-0.609289
